In [10]:
import cv2
import time
import numpy as np
import math
import mediapipe as mp
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume

################################
wCam, hCam = 640, 480                   #dimension of the camera window
################################


mpHands = mp.solutions.hands                             # we are importing the model that detects hands
hands = mpHands.Hands(min_detection_confidence=0.7)
mpDraw = mp.solutions.drawing_utils           #this draws on all the hands



devices = AudioUtilities.GetSpeakers()                        #this is used to create a connection between speaker and code
interface = devices.Activate(
    IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))
# volume.GetMute()
# volume.GetMasterVolumeLevel()
volRange = volume.GetVolumeRange()   # this gives us the range of the volume
minVol = volRange[0]
maxVol = volRange[1] 
vol = 0
volBar = 400
volPer = 0


cap = cv2.VideoCapture(0)
cap.set(3, wCam)
cap.set(4, hCam)
pTime = 0





while True:
    success, img = cap.read()
    
    
    #Detecting the Hand
    
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(imgRGB)
    # print(results.multi_hand_landmarks)

    lmList = []
    if results.multi_hand_landmarks:   # if a hand is detected
        for handLms in results.multi_hand_landmarks:   # traversing for each hang
            for id, lm in enumerate(handLms.landmark):
                # print(id, lm)
                h, w, c = img.shape     
                cx, cy = int(lm.x * w), int(lm.y * h)  # getting the value in pixels of the coordinnates of the 21 points on hands
                
                # print(id, cx, cy)
                
                lmList.append([id, cx, cy])  #this list will have the point and its coordinates
                
                # if id == 4:
                #     cv2.circle(img, (cx, cy), 15, (255, 0, 255), cv2.FILLED)
                    
                # if id == 8:
                #     cv2.circle(img, (cx, cy), 15, (255, 0, 255), cv2.FILLED)    
                
                 

            mpDraw.draw_landmarks(img, handLms, mpHands.HAND_CONNECTIONS) #this will draw the points
            
            
    
    if len(lmList) != 0:
        # print(lmList[4], lmList[8])

        x1, y1 = lmList[4][1], lmList[4][2]      #here we only need points of id 4 and 8 (index and thumb)
        x2, y2 = lmList[8][1], lmList[8][2]      
        cx, cy = (x1 + x2) // 2, (y1 + y2) // 2

        cv2.circle(img, (x1, y1), 15, (255, 0, 255), cv2.FILLED)       
        cv2.circle(img, (x2, y2), 15, (255, 0, 255), cv2.FILLED)
        cv2.line(img, (x1, y1), (x2, y2), (255, 0, 255), 3)
        cv2.circle(img, (cx, cy), 15, (255, 0, 255), cv2.FILLED)

        length = math.hypot(x2 - x1, y2 - y1)   #getting the distance between the two fingers
        # print(length)

        # Hand range 50 - 300
        # Volume Range -65 - 0  

        vol = np.interp(length, [50, 200], [minVol, maxVol])    #changing the length that is in the scale 50-300 to the scale of volume
        volBar = np.interp(length, [50, 200], [400, 150])
        volPer = np.interp(length, [50, 200], [0, 100])
        volume.SetMasterVolumeLevel(vol, None)   # this wills set the volume acc to the length

        if length < 50:
            cv2.circle(img, (cx, cy), 15, (0, 255, 0), cv2.FILLED)   

    cv2.rectangle(img, (50, 150), (85, 400), (255, 0, 0), 3)
    cv2.rectangle(img, (50, int(volBar)), (85, 400), (255, 0, 0), cv2.FILLED)
    cv2.putText(img, f'{int(volPer)} %', (40, 450), cv2.FONT_HERSHEY_COMPLEX,
                1, (255, 0, 0), 3)
            


    cTime = time.time()
    fps = 1 / (cTime - pTime)
    pTime = cTime
    cv2.putText(img, f'FPS: {int(fps)}', (40, 50), cv2.FONT_HERSHEY_COMPLEX, # to show FPS
                1, (255, 0, 0), 3)

    cv2.imshow("Img", img)
    
    if cv2.waitKey(1) & 0xFF == 27:
        break
    
cap.release()
cv2.destroyAllWindows()    

In [8]:
print(volume.GetVolumeRange())

(-63.5, 0.0, 0.5)
